In [2]:
import torch
import numpy as np

设，一个audio的所有帧（T）对应的phoneme ids是，并给出了audio长度为10：
```python　
predict_ids = torch.tensor([0, 0, 1, 1, 1, 2, 2, 2, 2, 2])
audio_lengths = [10]
```

那么，建立单项边时，有两个操作：
1. 添加所有邻接边：`0->1, 1->2, 2->3, ...`
2. 对于一个phoneme，对接下来的N个phoneme都建立边：
    - 以第1个node为例（index从0开始），设N=1，那么会新加边：`1->2, 1->3, 1->4`
    - 以第3个node为例（index从0开始），设N=1，那么会新加边：`3->5,6,7,8,9`

# Step 1 

使用arange可以很快地生成所有邻接边。

In [5]:
def get_adj_edges(L: int, use_np=False):
    if use_np:
        adj_edges = np.stack([np.arange(L - 1), np.arange(1, L)])
    else:
        adj_edges = torch.stack([torch.arange(L - 1), torch.arange(1, L)])
    return adj_edges

In [6]:
L = 149
adj_edges = get_adj_edges(L, use_np=0)
adj_edges_np = get_adj_edges(L, use_np=1)
print(adj_edges, adj_edges.numpy()-adj_edges_np)

tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
          28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
          42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
          56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
          70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
          84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
          98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
         112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
         126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
         140, 141, 142, 143, 144, 145, 146, 147],
        [  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
          15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
  

## Step 2

使用 `unique_consecutive`可以快速地查找所有不同的phoneme id，并定位其index范围。

### 遍历　

下面这种遍历长度L的方法，耗时太长了

#### torch

In [7]:
# predict_ids = torch.tensor([0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3])
predict_ids = torch.randint(0, 3, (149,))

N = 10

output, inverse, counts = predict_ids.unique_consecutive(return_inverse=True, return_counts=True)
cumsum_counts = torch.cumsum(counts, 0)
# output, inverse, conuts, cumsum_counts

edges = []
for i in range(L):
    unique_id = inverse[i]  # 0, 1, 2, 3,
    unique_id_end_index = cumsum_counts[unique_id]
    if unique_id == len(output) - 1:
        break
    next_id = min(len(output) - 1, unique_id + N)
    next_end_index = cumsum_counts[next_id]
    _edges = torch.stack(
        [torch.full((next_end_index - unique_id_end_index,), i), torch.arange(unique_id_end_index, next_end_index)]
    )
    edges.append(_edges)
edges = torch.concat(edges, 1)

In [8]:
edges

tensor([[  0,   0,   0,  ..., 145, 145, 145],
        [  1,   2,   3,  ..., 146, 147, 148]])

#### numpy 

首先，由于numpy中没有unique_consecutive这个函数，因此需要先实现他。

In [10]:
def unique_consecutive(x: np.ndarray):

    output = [x[0]]
    inverse = np.zeros_like(x)
    counts = [1]

    for i in range(1, len(x)):
        if x[i] == output[-1]:
            counts[-1] += 1
        else:
            output.append(x[i])
            counts.append(1)
        inverse[i] = len(output) -1
    return np.array(output), inverse, np.array(counts)

In [9]:
predict_ids = torch.tensor([0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3])
output, inverse, counts = predict_ids.unique_consecutive(return_inverse=True, return_counts=True)
output, inverse, counts

(tensor([0, 1, 2, 3]),
 tensor([0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3]),
 tensor([2, 3, 5, 2]))

In [ ]:
## 可以看到，和tensor版本的输出结果是一样的
unique_consecutive(predict_ids.numpy())

(array([0, 1, 2, 3]),
 array([0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3]),
 array([2, 3, 5, 2]))

In [205]:
L = 149 * 64
predict_ids = torch.randint(0, 3, (L,)).numpy()
N = 10

output, inverse, counts = unique_consecutive(predict_ids)
cumsum_counts = np.cumsum(counts, 0)
# output, inverse, conuts, cumsum_counts

edges = []
for i in range(L):
    unique_id = inverse[i]  # 0, 1, 2, 3,
    unique_id_end_index = cumsum_counts[unique_id]
    if unique_id == len(output) - 1:
        break
    next_id = min(len(output) - 1, unique_id + N)
    next_end_index = cumsum_counts[next_id]
    _edges = np.stack(
        [np.full((next_end_index - unique_id_end_index,), i), np.arange(unique_id_end_index, next_end_index)]
    )
    edges.append(_edges)
edges = np.concatenate(edges, 1)